In [1]:
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.model_selection import train_test_split

/home/ubuntu/Spell_Correction/Spell_Corrections/local/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
diabetes=pd.read_csv('/home/ubuntu/DEEP_LEARNING/Tensorflow_Deeplearning_02042018/02-TensorFlow-Basics/pima-indians-diabetes.csv')
print(diabetes.shape)
print(diabetes.columns)
(diabetes.head())

(768, 10)
Index([u'Number_pregnant', u'Glucose_concentration', u'Blood_pressure',
       u'Triceps', u'Insulin', u'BMI', u'Pedigree', u'Age', u'Class',
       u'Group'],
      dtype='object')


,Number_pregnant,Glucose_concentration,Blood_pressure,Triceps,Insulin,BMI,Pedigree,Age,Class,Group
0,6,0.743719,0.590164,0.353535,0.000000,0.500745,0.234415,50,1,B
1,1,0.427136,0.540984,0.292929,0.000000,0.396423,0.116567,31,0,C
2,8,0.919598,0.524590,0.000000,0.000000,0.347243,0.253629,32,1,B
3,1,0.447236,0.540984,0.232323,0.111111,0.418778,0.038002,21,0,B
4,0,0.688442,0.327869,0.353535,0.198582,0.642325,0.943638,33,1,C


In [3]:
cols_to_norm=[u'Number_pregnant', u'Glucose_concentration', u'Blood_pressure',u'Triceps', u'Insulin', u'BMI', u'Pedigree']
diabetes[cols_to_norm]=diabetes[cols_to_norm].apply(lambda x: (x-x.min())/(x.max()-x.min()))
print(diabetes.columns)
diabetes.head()

Index([u'Number_pregnant', u'Glucose_concentration', u'Blood_pressure',
       u'Triceps', u'Insulin', u'BMI', u'Pedigree', u'Age', u'Class',
       u'Group'],
      dtype='object')


,Number_pregnant,Glucose_concentration,Blood_pressure,Triceps,Insulin,BMI,Pedigree,Age,Class,Group
0,0.352941,0.743719,0.590164,0.353535,0.000000,0.500745,0.234415,50,1,B
1,0.058824,0.427136,0.540984,0.292929,0.000000,0.396423,0.116567,31,0,C
2,0.470588,0.919598,0.524590,0.000000,0.000000,0.347243,0.253629,32,1,B
3,0.058824,0.447236,0.540984,0.232323,0.111111,0.418778,0.038002,21,0,B
4,0.000000,0.688442,0.327869,0.353535,0.198582,0.642325,0.943638,33,1,C


In [4]:
num_preg=tf.feature_column.numeric_column('Number_pregnant')
gc=tf.feature_column.numeric_column('Glucose_concentration')
bp=tf.feature_column.numeric_column('Blood_pressure')
triceps=tf.feature_column.numeric_column('Triceps')
insulin=tf.feature_column.numeric_column('Insulin')
bmi=tf.feature_column.numeric_column('BMI')
Pedigree=tf.feature_column.numeric_column('Pedigree')
age=tf.feature_column.numeric_column('Age')

In [5]:
assign_group=tf.feature_column.categorical_column_with_vocabulary_list('Group',['A','B','C','D'])
#assign_group=tf.feature_column.categorical_column_with_hash_bucket('Group',hash_bucket_size=10)
age_bkt=tf.feature_column.bucketized_column(age,boundaries=[10,30,40,50,60,70,80])

In [6]:
feat_cols=[num_preg,gc,bp,triceps,insulin,bmi,Pedigree,age_bkt,assign_group]

In [7]:
X_data=diabetes.drop('Class',axis=1)
labels=diabetes['Class']
X_data.head()

,Number_pregnant,Glucose_concentration,Blood_pressure,Triceps,Insulin,BMI,Pedigree,Age,Group
0,0.352941,0.743719,0.590164,0.353535,0.000000,0.500745,0.234415,50,B
1,0.058824,0.427136,0.540984,0.292929,0.000000,0.396423,0.116567,31,C
2,0.470588,0.919598,0.524590,0.000000,0.000000,0.347243,0.253629,32,B
3,0.058824,0.447236,0.540984,0.232323,0.111111,0.418778,0.038002,21,B
4,0.000000,0.688442,0.327869,0.353535,0.198582,0.642325,0.943638,33,C


In [8]:
X_train, X_test, y_train, y_test = train_test_split(X_data, labels, test_size=0.33, random_state=42,stratify=labels)

In [9]:
input_func=tf.estimator.inputs.pandas_input_fn(x=X_train,y=y_train,batch_size=10,num_epochs=100,shuffle=True)

In [10]:
model=tf.estimator.LinearClassifier(feature_columns=feat_cols,n_classes=2)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_task_type': 'worker', '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7feb44082850>, '_save_checkpoints_steps': None, '_keep_checkpoint_every_n_hours': 10000, '_service': None, '_num_ps_replicas': 0, '_tf_random_seed': None, '_master': '', '_num_worker_replicas': 1, '_task_id': 0, '_log_step_count_steps': 100, '_model_dir': '/tmp/tmpZ12OBT', '_save_summary_steps': 100}


In [11]:
model.train(input_func,steps=1000)

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /tmp/tmpZ12OBT/model.ckpt.
INFO:tensorflow:loss = 6.931472, step = 1
INFO:tensorflow:global_step/sec: 198.672
INFO:tensorflow:loss = 5.2058663, step = 101 (0.513 sec)
INFO:tensorflow:global_step/sec: 230.313
INFO:tensorflow:loss = 6.0818405, step = 201 (0.430 sec)
INFO:tensorflow:global_step/sec: 239.706
INFO:tensorflow:loss = 5.3276834, step = 301 (0.420 sec)
INFO:tensorflow:global_step/sec: 246.039
INFO:tensorflow:loss = 2.8962238, step = 401 (0.401 sec)
INFO:tensorflow:global_step/sec: 233.509
INFO:tensorflow:loss = 6.4198236, step = 501 (0.430 sec)
INFO:tensorflow:global_step/sec: 234.653
INFO:tensorflow:loss = 8.243288, step = 601 (0.424 sec)
INFO:tensorflow:global_step/sec: 221.413
INFO:tensorflow:loss = 4.6574697, step = 701 (0.464 sec)
INFO:tensorflow:global_step/sec: 237.679
INFO:tensorflow:loss = 4.4379725, step = 801 (0.409 sec)
INFO:tensorflow:global_step/sec: 227.586
INFO:tensorflow:l

In [12]:
eval_input_func=tf.estimator.inputs.pandas_input_fn(x=X_test,y=y_test,batch_size=10,num_epochs=1,shuffle=False)

In [13]:
result=model.evaluate(eval_input_func)
result

INFO:tensorflow:Starting evaluation at 2018-04-05-08:04:58
INFO:tensorflow:Restoring parameters from /tmp/tmpZ12OBT/model.ckpt-1000
INFO:tensorflow:Finished evaluation at 2018-04-05-08:04:58
INFO:tensorflow:Saving dict for global step 1000: accuracy = 0.71653545, accuracy_baseline = 0.6496063, auc = 0.79257745, auc_precision_recall = 0.6869458, average_loss = 0.5244439, global_step = 1000, label/mean = 0.3503937, loss = 5.123414, prediction/mean = 0.35631895


{'accuracy': 0.71653545,
 'accuracy_baseline': 0.6496063,
 'auc': 0.79257745,
 'auc_precision_recall': 0.6869458,
 'average_loss': 0.5244439,
 'global_step': 1000,
 'label/mean': 0.3503937,
 'loss': 5.123414,
 'prediction/mean': 0.35631895}

In [14]:
pred_input_func=tf.estimator.inputs.pandas_input_fn(x=X_test,batch_size=10,num_epochs=1,shuffle=False)
predictions_=model.predict(pred_input_func)
list(predictions_)#=[pred for pred in predictions]

INFO:tensorflow:Restoring parameters from /tmp/tmpZ12OBT/model.ckpt-1000


[{'class_ids': array([0]),
  'classes': array(['0'], dtype=object),
  'logistic': array([0.18682954], dtype=float32),
  'logits': array([-1.4707441], dtype=float32),
  'probabilities': array([0.8131705 , 0.18682954], dtype=float32)},
 {'class_ids': array([0]),
  'classes': array(['0'], dtype=object),
  'logistic': array([0.47646096], dtype=float32),
  'logits': array([-0.09422565], dtype=float32),
  'probabilities': array([0.523539  , 0.47646102], dtype=float32)},
 {'class_ids': array([0]),
  'classes': array(['0'], dtype=object),
  'logistic': array([0.45623198], dtype=float32),
  'logits': array([-0.1755212], dtype=float32),
  'probabilities': array([0.543768, 0.456232], dtype=float32)},
 {'class_ids': array([0]),
  'classes': array(['0'], dtype=object),
  'logistic': array([0.40805373], dtype=float32),
  'logits': array([-0.37201703], dtype=float32),
  'probabilities': array([0.5919463 , 0.40805376], dtype=float32)},
 {'class_ids': array([0]),
  'classes': array(['0'], dtype=object)

In [15]:
embedded_group_col=tf.feature_column.embedding_column(assign_group,dimension=4)
feat_cols=[num_preg,gc,bp,triceps,insulin,bmi,Pedigree,age_bkt,embedded_group_col]

In [16]:
input_func=tf.estimator.inputs.pandas_input_fn(x=X_train,y=y_train,batch_size=10,num_epochs=100,shuffle=True)

In [17]:
dnn_model=tf.estimator.DNNClassifier(hidden_units=[10,20,20,20,10],feature_columns=feat_cols,n_classes=2)
dnn_model.train(input_func,steps=1000)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_task_type': 'worker', '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7feaf9512990>, '_save_checkpoints_steps': None, '_keep_checkpoint_every_n_hours': 10000, '_service': None, '_num_ps_replicas': 0, '_tf_random_seed': None, '_master': '', '_num_worker_replicas': 1, '_task_id': 0, '_log_step_count_steps': 100, '_model_dir': '/tmp/tmpa1cx3q', '_save_summary_steps': 100}
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /tmp/tmpa1cx3q/model.ckpt.
INFO:tensorflow:loss = 7.258046, step = 1
INFO:tensorflow:global_step/sec: 193.864
INFO:tensorflow:loss = 5.4588065, step = 101 (0.521 sec)
INFO:tensorflow:global_step/sec: 237.026
INFO:tensorflow:loss = 5.7609763, step = 201 (0.418 sec)
INFO:tensorflow:global_step/sec: 236.756
INFO:tensorflow:loss = 2.37

In [18]:
eval_input_func=tf.estimator.inputs.pandas_input_fn(x=X_test,y=y_test,batch_size=10,num_epochs=1,shuffle=False)
result=model.evaluate(eval_input_func)
result

INFO:tensorflow:Starting evaluation at 2018-04-05-08:05:09
INFO:tensorflow:Restoring parameters from /tmp/tmpZ12OBT/model.ckpt-1000
INFO:tensorflow:Finished evaluation at 2018-04-05-08:05:09
INFO:tensorflow:Saving dict for global step 1000: accuracy = 0.71653545, accuracy_baseline = 0.6496063, auc = 0.79257745, auc_precision_recall = 0.6869458, average_loss = 0.5244439, global_step = 1000, label/mean = 0.3503937, loss = 5.123414, prediction/mean = 0.35631895


{'accuracy': 0.71653545,
 'accuracy_baseline': 0.6496063,
 'auc': 0.79257745,
 'auc_precision_recall': 0.6869458,
 'average_loss': 0.5244439,
 'global_step': 1000,
 'label/mean': 0.3503937,
 'loss': 5.123414,
 'prediction/mean': 0.35631895}